# <div style="padding: 30px; color:white; margin:10; font-size:95%; text-align:left; display:fill; border-radius:10px; background-color:#3b3745"><b> Intel Review - Web Scraping - Techincal Reviews Scraper using Selenium</b></div>

<div style="background-color: #3b3745; border-radius: 12px; padding: 20px; box-shadow: 0 4px 8px 0 rgba(0,0,0,0.2);">
    <h2 style="color: #F1A424; text-align: center;">Table of Contents</h2>
    <ul style="list-style: none; padding: 0;">
        <li><a href="#section-1" style="color: white; text-decoration: none; display: flex; align-items: center; padding: 8px 15px; border-radius: 6px; transition: background-color 0.3s;"><span style="margin-right: 10px; font-weight: bold; color: #F1A424;">1.</span>  Importing Libraries</a></li>
        <li><a href="#section-2" style="color: white; text-decoration: none; display: flex; align-items: center; padding: 8px 15px; border-radius: 6px; transition: background-color 0.3s;"><span style="margin-right: 10px; font-weight: bold; color: #F1A424;">2.</span> Extracting links of Technical Reviews</a></li>
        <li><a href="#section-3" style="color: white; text-decoration: none; display: flex; align-items: center; padding: 8px 15px; border-radius: 6px; transition: background-color 0.3s;"><span style="margin-right: 10px; font-weight: bold; color: #F1A424;">3.</span>   Scraping Technical Reviews </a></li>
    </ul>
</div>

<a id="section-1"></a>
# <div style="padding: 30px; color:white; margin:10; font-size:75%; text-align:left; display:fill; border-radius:10px; background-color:#3b3745"><b><span style='color:#F1A424'>1 |</span></b> <b>  Importing Libraries</b></div>

In [1]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver import Firefox
import pandas as pd
from selenium import webdriver
import time

In [2]:
%pip install webdriver-manager


Note: you may need to restart the kernel to use updated packages.


<a id="section-2"></a>
# <div style="padding: 30px; color:white; margin:10; font-size:75%; text-align:left; display:fill; border-radius:10px; background-color:#3b3745"><b><span style='color:#F1A424'>2 |</span></b> <b> Extracting links of Techical Reviews</b></div>

In [ ]:
filename = 'processors.txt'

with open(filename, 'r') as file:
    processor_names = [line.strip() for line in file.readlines()]

driver = Firefox()
driver.maximize_window()

# Test search query
test_query = 'test query'
try:
    driver.get(f'https://www.google.com/search?q={test_query}')
    
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'h3')))
    
    print("Google search test successful.")
except Exception as ex:
    print(f"Error during Google search test: {ex}")
    driver.quit()
    exit()

results = []

# Websites to search
websites = {
    'tomshardware': 'site:tomshardware.com',
    'pcmag': 'site:pcmag.com'
}

for processor_name in processor_names:
    processor_name_cleaned = processor_name.replace(' ', '+')
    
    for site, site_keyword in websites.items():
        search_query = f'{processor_name_cleaned}+{site_keyword}+review'
        try:
            driver.get(f'https://www.google.com/search?q={search_query}')
            
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'h3')))
            
            search_results = driver.find_elements(By.CSS_SELECTOR, 'h3')
            first_result = search_results[0]
            link = first_result.find_element(By.XPATH, './../../a').get_attribute('href')
            
            results.append({
                'Processor': processor_name,
                'Source': site,
                'URL': link
            })
            
            print(f"Found {site} review for {processor_name}")
        except Exception as ex:
            print(f"Error processing {site} review for {processor_name}: {ex}")
    
# Saving results to CSV
df = pd.DataFrame(results)
df.to_csv('processors_links.csv', index=False)

driver.quit()
print("Finished")


<a id="section-3"></a>
# <div style="padding: 30px; color:white; margin:10; font-size:75%; text-align:left; display:fill; border-radius:10px; background-color:#3b3745"><b><span style='color:#F1A424'>3 |</span></b> <b> Scraping Technical Reviews</b></div>

In [ ]:
driver = Firefox()
driver.maximize_window()

# Function to scrape content from tomshardware.com
def scrape_tomshardware(url):
    driver.get(url)
    time.sleep(15)  # Wait for the page to load completely
    main_content = driver.find_element(By.ID, 'article-body')
    # Extract the paragraphs that are not inside figure or aside tags and do not contain a tags
    paragraphs = main_content.find_elements(By.XPATH, './/p[not(ancestor::figure) and not(ancestor::aside) and not(descendant::a)]')
    # Join the text content of the paragraphs
    content = ' '.join([para.text for para in paragraphs])
    return content

# Function to scrape content from pcgamer.com
def scrape_pcmag(url):
    driver.get(url)
    time.sleep(5)  
    content_container = driver.find_element(By.ID, 'article')  
    paragraphs = content_container.find_elements(By.TAG_NAME, 'p')
    content = ' '.join([para.text for para in paragraphs])
    return content

# Read input CSV file
input_csv = 'processors_links.csv'
df = pd.read_csv(input_csv)

results = []

for index, row in df.iterrows():
    processor = row['Processor']
    source = row['Source']
    url = row['URL']
    
    if source == 'tomshardware':
        content = scrape_tomshardware(url)
    elif source == 'pcmag':
        content = scrape_pcmag(url)
    else:
        content = f"Scraping function not defined for source: {source}"
    
    results.append({'Processor': processor, 'Source': source, 'Content': content})

driver.quit()

results_df = pd.DataFrame(results)

# Save the results to a new CSV file
output_csv = 'scraped_processors.csv'
results_df.to_csv(output_csv, index=False)

print(f"Scraping completed. Data saved to {output_csv}")


                                   This concludes the analysis presented in this notebook.
